# 청와대 청원 크롤링 코드 

## 작성자: 박하람
## 작성일자: 2020년 8월 10일 

분야별 청원 전체 카테고리 선택 후, 추천 순 정렬하여 모든 청원 가져오기

In [1]:
import sys
sys.path.append("/usr/local/lib/python3.8/site-packages")

In [118]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd
import time

In [217]:
def get_peti_df(start, end): 
    
    category = []
    titles = []
    address = []
    end_date = []
    consent_num = []
    
    for n in tqdm(range(start, end+1)): 
        url = f'https://www1.president.go.kr/petitions/?c=0&only=2&page={n}&order=2'
        soup = BeautifulSoup(urlopen(url), "html.parser")
        
        if n % 2 != 0: 
            time.sleep(1)
        else: 
            time.sleep(0.5)
            
        contents = soup.find_all('div', class_='bl_body')
        
        for i in range(7):
            time.sleep(0.5)
            peti_list = contents[0].find_all('div', class_='bl_wrap')[i]
            
            # 청원 카테고리
            category.append(peti_list.find('div', class_='bl_category').get_text()) 
            # 청원 제목 
            titles_raw = peti_list.find('div', class_='bl_subject').get_text()
            titles_split = re.split('\r|\t|\n|\xa0|제목', titles_raw)
            titles.append(''.join(titles_split))
            # 청원 주소 
            add_url = peti_list.find('a')['href']
            address.append('https://www1.president.go.kr'+add_url)
            # 청원 만료일 
            end_date.append(peti_list.find('div', class_='bl_date').get_text())
            # 청원 동의 수 
            consent_num_raw = peti_list.find('div', class_='bl_agree').get_text()
            consent_split = re.split('\r|\t|\n|\xa0', consent_num_raw)
            consent_num.append(''.join(consent_split).split()[1][:-1])
        
    petition_dict = {'카테고리':category, '제목':titles, '청원종료일': end_date, 
                 '참여인원': consent_num,'URL': address}
    df = pd.DataFrame(petition_dict)
    return df 
        

In [209]:
%time peti_df_1 = get_peti_df(1,50)

100%|██████████| 50/50 [00:17<00:00,  2.89it/s]

CPU times: user 4.36 s, sys: 157 ms, total: 4.52 s
Wall time: 17.3 s


In [218]:
%time peti_df_2 = get_peti_df(51,100)

  0%|          | 0/50 [00:01<?, ?it/s]


IndexError: list index out of range

In [159]:
%time peti_df_3 = get_peti_df(101,150)

100%|██████████| 50/50 [00:50<00:00,  1.02s/it]

CPU times: user 3.74 s, sys: 122 ms, total: 3.86 s
Wall time: 51 s


In [160]:
%time peti_df_4 = get_peti_df(151,200)

100%|██████████| 50/50 [00:50<00:00,  1.02s/it]

CPU times: user 3.76 s, sys: 113 ms, total: 3.87 s
Wall time: 51 s


In [167]:
%time peti_df_5 = get_peti_df(201,250)

100%|██████████| 50/50 [06:02<00:00,  7.25s/it]

CPU times: user 4.08 s, sys: 134 ms, total: 4.22 s
Wall time: 6min 2s


In [174]:
%time peti_df_6 = get_peti_df(251, 300)

100%|██████████| 50/50 [02:51<00:00,  3.43s/it]

CPU times: user 4.25 s, sys: 157 ms, total: 4.41 s
Wall time: 2min 51s


In [175]:
peti_df = pd.concat([peti_df_1, peti_df_2, peti_df_3, peti_df_4, peti_df_5, peti_df_6])
len(peti_df)

2100

In [178]:
peti_df.head()

,카테고리,제목,청원종료일,참여인원,URL
0,일자리,제목 공기업 비정규직의 정규화 그만해주십오.,20.07.23,"352,266",https://www1.president.go.kr/petitions/589941
1,인권/성평등,제목 '저의 딸이 강간 당하는 것을 목격하여..' 그 충격과 고통으로 딸이 평생 ...,20.07.11,"286,148",https://www1.president.go.kr/petitions/589634
2,안전/환경,제목 응급환자가 있는 구급차를 막아세운 택시 기사를 처벌해 주세요.,20.08.02,"735,972",https://www1.president.go.kr/petitions/590341
3,인권/성평등,제목 강영수 서울고법 수석부장판사의 대법관 후보 자격 박탈을 청원합니다.,20.08.05,"529,144",https://www1.president.go.kr/petitions/590416
4,행정,제목 정부의 교회 정규 예배 이외 행사 금지를 취소해주세요.,20.08.07,"427,470",https://www1.president.go.kr/petitions/590504
